In [ ]:
#Quick cell to make jupyter notebook use the full screen wi"dth
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Basic utility functions
import logging
import pathlib
from src.logging import logger
from src.paths import Paths
from src.utils import list_dir
paths = Paths()

In [ ]:
from src.data import DataSource, Dataset
from src import workflow

### Color by SRM

We are going to look at beer so let's see if we can color by SRM.  SRM is a means by which brewers estimate color of the beer.  We'll grab a table of SRM values for beer styes and use it to augment our data.

Let's make the source of these files urls from github.

In [ ]:
srm_dsrc = DataSource('beer_styles_srm')

In [ ]:
!ls -la ../../EmbedAllTheThings/

Getting html headers in things. Don't want to mess with this anymore. I'm just going to grab the files directly. 

In [ ]:
# Add requisite files
# stem = 'https://raw.githubusercontent.com/jc-healy/EmbedAllTheThings/4d26897b4b9412ed5c9bae2cb1c472b9d3137fc4/'
source = pathlib.Path('../../EmbedAllTheThings/').resolve()
file_names = ['beer_styles.csv', 'srm2rgb.csv', 'beer_review_kaggle2srm_schema.csv']

for name in file_names:
    srm_dsrc.add_file(source_file=source/name)

In [ ]:
srm_dsrc.file_list

In [ ]:
# Add metadata...essentially, there is none.

metadata_txt = """
This data is from https://github.com/jc-healy/EmbedAllTheThings. No previous source or \
license is available. No license on the repo either."""

srm_dsrc.add_metadata(contents=metadata_txt, kind='DESCR')

In [ ]:
srm_dsrc.fetch()

In [ ]:
!ls -la $paths.raw_data_path

In [ ]:
srm_dsrc.unpack()


    srm_raw = pd.read_csv('beer_styles.csv')
    schema_match = pd.read_csv('beer_review_kaggle2srm_schema.csv')
    srm = schema_match.merge(srm_raw, how='left', right_on='Style Name', left_on='srm_style')

    srm2rgb = pd.read_csv('srm2rgb.csv', header=None, sep=' ')
    srm2rgb.columns = ['srm_rgb','srm']
    srm_data = srm.merge(srm2rgb, how='left', left_on=srm['SRM Mid'].apply(int), right_on='srm').drop(['srm'], axis =1)
    #srm_data.to_csv('beer_style_enriched.csv', index=False)

# Let's process this

In [ ]:
import pandas as pd
import pathlib

def process_srm_data(*, source_dir, kind='all', metadata=None):
    """
    Process beer reviews into (data, target, metadata) format. Since we plan to use Pandas
    for further processing, data will be a pandas dataframe. As Pandas will read that too.
    
    In this case, if we 
    
    Parameters
    ----------
    source_dir:
        The directory the reviews have been put in (likely the raw data dir)
    raw_dir:
    kind: {'all'}
        This is an unsupervised learning example. There are no labels. We will only work
        with the whole dataset. (Optionally add train and test set later for experimenting.)
        
    
    Returns
    -------
    A tuple:
        (data, target, additional_metadata)
        
    """
    if metadata is None:
        metadata = {}
    
    if source_dir:
        source_dir = pathlib.Path(source_dir)
    
    srm_raw = pd.read_csv(source_dir / 'beer_styles.csv')
    schema_match = pd.read_csv(source_dir / 'beer_review_kaggle2srm_schema.csv')
    srm = schema_match.merge(srm_raw, how='left', right_on='Style Name', left_on='srm_style')

    srm2rgb = pd.read_csv(source_dir / 'srm2rgb.csv', header=None, sep=' ')
    srm2rgb.columns = ['srm_rgb','srm']
    data = srm.merge(srm2rgb, how='left', left_on=srm['SRM Mid'].apply(int), right_on='srm').drop(['srm'], axis =1)

    target = None
    
    return data, target, metadata

In [ ]:
# test it 
data, target, metadata = process_srm_data(source_dir=paths.raw_data_path)

## Added it to src directly

In [ ]:
from src.data.localdata import process_srm_data

In [ ]:
help(process_srm_data)

In [ ]:
from functools import partial

In [ ]:
srm_dsrc.parse_function = partial(process_srm_data, source_dir=str(paths.raw_data_path))

In [ ]:
ds = srm_dsrc.process()

In [ ]:
ds

In [ ]:
workflow.available_datasources()

In [ ]:
workflow.add_datasource(srm_dsrc)

In [ ]:
workflow.available_datasources()

In [ ]:
srm_ds = Dataset.from_datasource('beer_styles_srm')

In [ ]:
srm_ds

In [ ]:
workflow.available_datasources()

In [ ]:
workflow.add_transformer(from_datasource='beer_styles_srm',
                         output_dataset='beer_styles_srm_all')

In [ ]:
workflow.make_data()

In [ ]:
workflow.available_datasets()

In [ ]:
ds = Dataset.load('beer_styles_srm_all')

In [ ]:
ds